# Second order PLL

Reference pg175 of Tsui

## Initialization of variables

In [28]:
import numpy as np
import control
import controlsystems as cs
from scipy import signal
import matplotlib.pyplot as plt

# Initializations

phi_hat = 0.3      # phi_hat is the output phase 
freq_offset = 0.3
delta_phi = 0.0    # delta_phi is the phase difference between the incomming signal and the VCO


noise_power = 0
noise_mean = 0


Bnd = 1 #Hz
Bnc = 20 #Hz
Kd = 50
Kc = 4 * np.pi * 100
fs = 4092000
ts = 1/fs
N = fs       # Number of samples
# These are just initializing some vectors for plotting
ref = np.linspace(0, N, N)
Zeta = 0.707
wnc = (2*Bnc) / (Zeta + 1/(4*Zeta))
wnd = (2*Bnd) / (Zeta + 1/(4*Zeta))

carrC1 = (8*Zeta*wnc*ts) / (Kc*(4 + 4*Zeta*wnc*ts + (wnc*ts)**2))
carrC2 = (4*(wnc*ts)**2) / (Kc*(4 + 4*Zeta*wnc*ts + (wnc*ts)**2))

codeC1 = (8*Zeta*wnd*ts) / (Kd*(4 + 4*Zeta*wnd*ts + (wnd*ts)**2))
codeC2 = (4*(wnd*ts)**2) / (Kd*(4 + 4*Zeta*wnd*ts + (wnd*ts)**2))

print(carrC1)
print(carrC2)
print(codeC1)
print(codeC2)

1.0370666751536263e-08
6.759697056306955e-14
1.3032244865276326e-09
4.247269214384406e-16


# Create z-domain transfer functions H(z) 

In [23]:
carr_num = np.array([Kc*(carrC1 + carrC2),  -Kc*(carrC1)])
carr_den = np.array([1, Kc*(carrC1 + carrC2) - 2, 1 - Kc*carrC1])

code_num = np.array([Kd*(codeC1 + codeC2),  -Kd*(codeC1)])
code_den = np.array([1, Kd*(codeC1 + codeC2) - 2, 1 - Kd*codeC1])

carrH = control.tf(carr_num, carr_den, ts)
print()
print(carrH)

codeH = control.tf(code_num, code_den, ts)
print()
print(codeH)



1.303e-05 z - 1.303e-05
-----------------------
     z^2 - 2 z + 1

dt = 2.4437927663734114e-07



6.516e-07 z - 6.516e-07
-----------------------
     z^2 - 2 z + 1

dt = 2.4437927663734114e-07



# Create matrix representation of H(z)

In [24]:
carrHmatrix = np.matrix(carr_num, carr_den)
print(carr_num[1])

TypeError: data type not understood

# Calculate tau values

In [25]:
tau1 = ts/carrC2
tau2 = (2*tau1*carrC1 + ts)/2

print(tau1)
print(tau2)

3615240.0706970966
0.03749257218963831


## Generate a sine wave to test with

This block generates a simple sine wave which is set by the phi and freq_offset parameters. 

In [26]:
phi = 0.0              # Original signal phase
def tstSignalGen(ref, phi, freq_offset):
    signal = []       # Save the signal_in for plotting
    for i in ref:

        # Complex input signal
        signal_in = np.cos(phi) + 1j*np.sin(phi)
        phi += freq_offset

        signal.append(signal_in) # Saved for plotting
    return signal

# Create input signal X(z)

In [29]:
#noise = np.random.normal(noise_mean, noise_power, N)
inputsignal = np.array(tstSignalGen(ref, phi, freq_offset)) #+ noise

carr_num = [Kc*(carrC1 + carrC2),  Kc*(carrC1)]
carr_den = [1, Kc*(carrC1 + carrC2) - 2, 1 - Kc*carrC1]

code_num = [Kd*(codeC1 + codeC2),  Kd*(codeC1)]
code_den = [1, Kd*(codeC1 + codeC2) - 2, 1 - Kd*codeC1]

X = control.tf(carr_num, carr_den, ts)
print(X)

margins = control.stability_margins(X, True)
print(margins)
#(T, yout, xout) = control.step_response(X, ref)
print(len(ref))
#lockedsig = phaseLock(inputsignal, phi_hat, delta_phi)


1.303e-05 z + 1.303e-05
-----------------------
     z^2 - 2 z + 1

dt = 2.4437927663734114e-07

(3.2590431307706234e-06, None, None, 1.7320395202103638, None, None)
4092000


# Calculate output by multiplying X(z)* H(z) = Y(z)


In [8]:
carrY = X * carrH 
codeY = X * codeH

print(carrY)
print(codeY)



1.303e-05 z^3 - 1.514e-06 z^2 - 1.152e-05 z
-------------------------------------------
z^4 - 0.2323 z^3 - 1.535 z^2 - 0.2323 z + 1

dt = 2.443792766373412e-07


6.516e-07 z^3 - 7.569e-08 z^2 - 5.759e-07 z
-------------------------------------------
z^4 - 0.2323 z^3 - 1.535 z^2 - 0.2323 z + 1

dt = 2.443792766373412e-07



In [ ]:
print(yout)

# Convert output from z-domain to t-domain

In [9]:
T, yout = control.impulse_response(carrH)


#plt.plot(response, ts)
#plt.show()

KeyboardInterrupt: 

# Plot real & imaginary outputs

In [ ]:
#plt.subplot(2, 1, 1)
plt.plot(ref, np.real(inputsignal), color = 'lightblue')
#plt.plot(ref, np.real(lockedsig), color = 'firebrick')



#plt.subplot(2, 1, 2)
#plt.plot(ref, np.imag(lockedsig))
#plt.xlabel("Samples")

#plt.ylabel("Amplitude")
plt.show()

## 2 signal comparison

Tested 2 different signals for direct comparison

In [ ]:
in1 = tstSignalGen(ref, 0.0, 0.4)
in2 = tstSignalGen(ref, 0.0, 0.09)

lock1 = phaseLock(in1, phi_hat, delta_phi)
lock2 = phaseLock(in2, phi_hat, delta_phi)

In [ ]:
plt.plot(np.real(in1))
plt.plot(np.real(lock1))
plt.show()
plt.plot(np.real(in2))
plt.plot(ref, np.real(lock2))
plt.show()

